# Setting Up

In [ ]:
%pip install ultralytics

In [ ]:
# Update settings for defining project directories
from ultralytics import settings
import os

HOME = os.getcwd()

settings.update({"datasets_dir": f"{HOME}\\datasets"})
settings.update({"weights_dir": f"{HOME}\\weights"})
settings.update({"runs_dir": f"{HOME}\\runs"})
print(settings)

# Training a Predefined Model

Mandatory steps:
- Create weights, datasets and predictions directories inside HOME directory (ultralytics directory)
- Download weights from [ultralytics website](https://docs.ultralytics.com/tasks/segment/#models) and leave them inside /ultralytics/weights/
- Properly define your dataset and your config.yaml file
- Create image and video directories inside /ultralytics/predictions/

## Training & Validation

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n-seg.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(
    task="segment",
    mode="train",
    model=model,
    data=f"{HOME}\\datasets\\config.yaml",
    epochs=10,
    batch=16,
    workers=8,
    close_mosaic=10,
    imgsz=416,
    patience=0)

In [ ]:
exp_PATH = f'{HOME}/runs/segment/train10/weights/best.pt'     # Change your model folder!
os.path.exists(exp_PATH)

In [ ]:
# Validate the model using the 'coco8.yaml' dataset for 3 epochs
results = model.val(
    task="segment",
    mode="val",
    model=exp_PATH,
    data=f"{HOME}/datasets/config.yaml",
    project=f"{HOME}/runs/segment/train10",          # Change your project folder!
    name="validation")

## Prediction

In [60]:
# Load your model
model = YOLO(f"{HOME}/runs/segment/train10/weights/best.pt")

In [ ]:
# Pick up a random test image
import random

images_directory_path = f"{HOME}/datasets/split/test/images"

# List all image files in the directory and pick a random image file
image_files = [f for f in os.listdir(images_directory_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
random_image_file = random.choice(image_files)

# Define the full path to the random image
random_image_path = os.path.join(images_directory_path, random_image_file)
print(random_image_path)

In [ ]:
results = model(f'{random_image_path}')

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename=f"{HOME}/predictions/images/{random_image_file}")  # save to disk

In [ ]:
# Display last predicted image
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

image_file = f"{HOME}/predictions/images/{random_image_file}"

# Load the image using OpenCV
image = cv2.imread(image_file)

# Convert the image from BGR to RGB for display purposes
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.figure(figsize=(3, 3))
plt.imshow(image_rgb)
plt.title(f'{random_image_file}')
plt.axis('off')
plt.show()